In [1]:
pip install sqlite3


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [2]:
import sqlite3

# Connect to SQLite (or create database if it doesn't exist)
conn = sqlite3.connect("news.db")
cursor = conn.cursor()

# Create a table for storing headlines
cursor.execute("""
CREATE TABLE IF NOT EXISTS headlines (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    source TEXT
)
""")

conn.commit()
conn.close()

print("✅ Database and table created successfully!")


✅ Database and table created successfully!


In [3]:
import requests
from bs4 import BeautifulSoup
import sqlite3

# Define the news website
URL = "https://www.bbc.com/news"

# Fetch the webpage
response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")

# Find all headlines
headlines = soup.find_all("h2")

# Connect to SQLite database
conn = sqlite3.connect("news.db")
cursor = conn.cursor()

# Insert each headline into the database
for headline in headlines[:10]:  # Limit to 10 headlines
    title = headline.text.strip()
    cursor.execute("INSERT INTO headlines (title, source) VALUES (?, ?)", (title, "BBC"))

conn.commit()
conn.close()

print("✅ News headlines stored successfully!")


✅ News headlines stored successfully!


In [4]:
# Connect to the database
conn = sqlite3.connect("news.db")
cursor = conn.cursor()

# Retrieve all stored headlines
cursor.execute("SELECT * FROM headlines")
rows = cursor.fetchall()

# Display the results
print("\n🔍 Stored News Headlines:")
for row in rows:
    print(f"{row[0]}. {row[1]} (Source: {row[2]})")

conn.close()



🔍 Stored News Headlines:
1. Chinese AI chatbot DeepSeek sparks market turmoil (Source: BBC)
2. Is China's AI tool DeepSeek as good as it seems? (Source: BBC)
3. Belgian footballer Nainggolan arrested in cocaine trafficking sting (Source: BBC)
4. Holocaust survivors recall horrors of Auschwitz as Prince William and Kate attend London memorial (Source: BBC)
5. Moment car accidentally drives into Philadelphia Eagles fans (Source: BBC)
6. Is China's AI tool DeepSeek as good as it seems? (Source: BBC)
7. Trump to sign order eliminating DEI from military (Source: BBC)
8. Israel says eight hostages due to be freed in first phase are dead (Source: BBC)
9. Palestinians return to north Gaza on foot, with belongings in hand (Source: BBC)
10. Belgian footballer Nainggolan arrested in cocaine trafficking sting (Source: BBC)


In [9]:
conn = sqlite3.connect("news.db")
cursor = conn.cursor()

# Add the column only if it doesn't exist
try:
    cursor.execute("ALTER TABLE headlines ADD COLUMN date_scraped TEXT")
    conn.commit()
    print("✅ Column 'date_scraped' added successfully.")
except sqlite3.OperationalError:
    print("⚠️ Column 'date_scraped' already exists.")

conn.close()


✅ Column 'date_scraped' added successfully.


In [10]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import datetime

# Define news sources
SOURCES = {
    "BBC": "https://www.bbc.com/news",
    "CNN": "https://edition.cnn.com/world"
}

# Connect to SQLite database
conn = sqlite3.connect("news.db")
cursor = conn.cursor()

# Ensure the table exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS headlines (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    source TEXT,
    date_scraped TEXT
)
""")

# Function to scrape and store news
def scrape_news():
    for source, url in SOURCES.items():
        print(f"🔍 Scraping {source}...")

        # Fetch webpage
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        # Find headlines (adjust as needed for CNN structure)
        if source == "BBC":
            headlines = soup.find_all("h2")
        elif source == "CNN":
            headlines = soup.find_all("span", class_="container__headline-text")

        # Store headlines in the database
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        for headline in headlines[:10]:  # Limit to top 10 headlines
            title = headline.text.strip()
            cursor.execute("INSERT INTO headlines (title, source, date_scraped) VALUES (?, ?, ?)", 
                           (title, source, timestamp))

    conn.commit()
    print("✅ News headlines stored successfully!")

# Run the scraper
scrape_news()

# Close database connection
conn.close()


🔍 Scraping BBC...
🔍 Scraping CNN...
✅ News headlines stored successfully!


In [15]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect("news.db")

# Load data
df = pd.read_sql("SELECT * FROM headlines", conn)
conn.close()

# Display first few rows
print(df.head(5))


   id                                              title source date_scraped
0   1  Chinese AI chatbot DeepSeek sparks market turmoil    BBC         None
1   2   Is China's AI tool DeepSeek as good as it seems?    BBC         None
2   3  Belgian footballer Nainggolan arrested in coca...    BBC         None
3   4  Holocaust survivors recall horrors of Auschwit...    BBC         None
4   5  Moment car accidentally drives into Philadelph...    BBC         None


In [16]:
import sqlite3

conn = sqlite3.connect("news.db")
cursor = conn.cursor()

# List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

conn.close()

print("📂 Tables in the database:", tables)


📂 Tables in the database: [('headlines',), ('sqlite_sequence',)]
